# load data

In [1]:
import addict
import pandas as pd
# from data_io import load_train_features, load_test_features
from data_io import load_train_all_features, load_test_all_features
from validator import KFoldValidator
from models import LGBModel
from my_logger import print_info, init_global_logger


def encode_feature123(data, encoder, is_train=True):
    if is_train:
        data["outlier"] = (data.target < -30)
        encoder = addict.Dict()
    for col in ["feature_1", "feature_2", "feature_3"]:
        if is_train:
            encoder[col] = data.groupby([col])['outlier'].mean()
        data[col] = data[col].map(encoder[col])
    return data, encoder


model_name = "notebook022"
init_global_logger("train_predict_" + model_name)
train_data = load_train_all_features()
print_info("train_data.shape", train_data.shape)
print_info("train_data.head", train_data.head())
train_data, encoder_ = encode_feature123(train_data, None, is_train=True)

# features = list(train_data.columns)
# features.remove("outlier")
# features.remove("card_id")
# features.remove("target")

features = list(pd.read_csv("./models/lgb018.csv").feature_name.values)

test_data = load_test_all_features()
test_data, encoder_ = encode_feature123(test_data, encoder_, is_train=False)

train - done in 0s
all_trans - done in 0s
hist_trans - done in 6s
new_trans - done in 8s
newk - done in 1s
psum - done in 1s
monthly_pmax - done in 1s
monthly_merchant_pmax - done in 1s
main_merchant_count - done in 1s
monthly_merchant_avg_std - done in 1s
2019-02-18 11:21:58 train_predict_notebook019 >>> [info] train_data.shape : (201917, 1177)
2019-02-18 11:21:58 train_predict_notebook019 >>> [info] train_data.head :    feature_1  feature_2  feature_3    target          card_id  fa_month  \
0        5.0        2.0        1.0 -0.820283  C_ID_92a2005557  201706.0   
1        4.0        1.0        0.0  0.392913  C_ID_3d0044924f  201701.0   
2        2.0        2.0        0.0  0.688056  C_ID_d639edf6cd  201608.0   
3        4.0        3.0        0.0  0.142495  C_ID_186d6a6901  201709.0   
4        1.0        3.0        0.0 -0.159749  C_ID_cdbd2c0db2  201711.0   

   purchase_amount_sum_hist_trans  purchase_amount_mean_hist_trans  \
0                     -165.968735                       

In [7]:
train_dtypes = train_data.dtypes
features = list(train_dtypes[(train_dtypes == 'float32') | (train_dtypes == 'float64')].index)

In [13]:
features.remove("target")

In [9]:
from tqdm import tqdm_notebook

In [14]:
from scipy.stats import ks_2samp
from tqdm import tqdm

list_p_value =[]

for ftr in tqdm_notebook(features):
    list_p_value.append(ks_2samp(test_data[ftr] , train_data[ftr])[1])

In [15]:
train_test_cmp = pd.Series(list_p_value, index=features).sort_values() 

In [21]:
discard = list(train_test_cmp[train_test_cmp < .1].index)

In [23]:
sorted(discard)

['category_2_3.0_hist_trans',
 'newk_amount_month_ratio_max',
 'newk_duration_max',
 'newk_duration_mean',
 'newk_hist_amount_month_ratio_max',
 'newk_hist_category_3_mean',
 'newk_hist_installments_mean',
 'newk_hist_price_max',
 'newk_hist_price_mean',
 'newk_hist_price_sum',
 'newk_hist_purchase_amount_max',
 'newk_installments_mean',
 'newk_new_category_2_mean',
 'newk_new_category_2_mean_mean',
 'newk_new_duration_mean',
 'newk_new_hour_mean',
 'newk_new_hour_min',
 'newk_new_price_mean',
 'newk_new_purchase_amount_mean',
 'newk_price_max',
 'newk_price_mean',
 'newk_price_total',
 'newk_purchase_amount_max',
 'newk_purchase_amount_mean',
 'purchase_amount_max_hist_trans',
 'purchase_amount_mean_new_trans',
 'purchase_amount_sum_new_trans',
 'ratio_1_lag-4_monthly_merchant_avg_std',
 'ratio_1_lag0_psum',
 'sum_1_lag-4_psum',
 'sum_1_lag1_monthly_pmax',
 'sum_1_lag2_monthly_merchant_pmax',
 'sum_1_lag2_monthly_pmax',
 'sum_2_lag-4_monthly_merchant_pmax',
 'sum_2_lag1_monthly_mercha